In [ ]:
!pip install flwr
# !kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection
# !unzip brain-mri-images-for-brain-tumor-detection.zip
# pip install torch torchvision
# !pip install -U flwr[simulation]
# pip install "ray[default]" --ignore-installed


In [ ]:
import os
import shutil
import zipfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import ndarrays_to_parameters, Context
from flwr.server import ServerApp, ServerConfig
from flwr.server import ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
import flwr as fl
from flwr.client import NumPyClient


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import flwr as fl
from flwr.server.strategy import FedAvg

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root="/content/brain_tumor_dataset", transform=transform)

print(f"Number of samples: {len(dataset)}")
print(f"Classes: {dataset.classes}")
print(f"Class-to-Index Mapping: {dataset.class_to_idx}")

total_length = len(dataset)
split_size = total_length // 5
torch.manual_seed(42)

split_sizes = [split_size, split_size, split_size, split_size, total_length - 4 * split_size]
part1, part2, part3, part4, part5 = random_split(dataset, split_sizes)
train_sets = [part1, part2, part3, part4, part5]

def build_model(num_classes):
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    top_model = base_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(1, activation='sigmoid')(top_model) if num_classes == 2 else Dense(num_classes, activation='softmax')(top_model)

    model = Model(inputs=base_model.input, outputs=top_model)
    model.compile(loss='binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    return model

Number of samples: 253
Classes: ['no', 'yes']
Class-to-Index Mapping: {'no': 0, 'yes': 1}


In [ ]:
def build_model(num_classes):
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    top_model = base_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(1, activation='sigmoid')(top_model) if num_classes == 2 else Dense(num_classes, activation='softmax')(top_model)

    model = Model(inputs=base_model.input, outputs=top_model)
    model.compile(loss='binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    return model

In [ ]:
# Sets the parameters of the model
def set_weights(net, parameters):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict(
        {k: torch.tensor(v) for k, v in params_dict}
    )
    net.load_state_dict(state_dict, strict=True)

# Retrieves the parameters from the model
def get_weights(net):
    ndarrays = [
        val.cpu().numpy() for _, val in net.state_dict().items()
    ]
    return ndarrays

In [ ]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainset, testset):
        self.net = net
        self.trainset = trainset
        self.testset = testset

    # Train the model
    def fit(self, parameters, config):
        set_weights(self.net, parameters)
        train_model(self.net, self.trainset)
        return get_weights(self.net), len(self.trainset), {}

    # Test the model
    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
        set_weights(self.net, parameters)
        loss, accuracy = evaluate_model(self.net, self.testset)
        return loss, len(self.testset), {"accuracy": accuracy}

NameError: name 'Dict' is not defined

In [ ]:
# Client function
def client_fn(context: Context) -> Client:
    net = SimpleModel()
    partition_id = int(context.node_config["partition-id"])
    client_train = train_sets[int(partition_id)]
    client_test = testset
    return FlowerClient(net, client_train, client_test).to_client()

In [ ]:
client = ClientApp(client_fn)

In [ ]:
net = SimpleModel()
params = ndarrays_to_parameters(get_weights(net))

def server_fn(context: Context):
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        initial_parameters=params,
        evaluate_fn=evaluate,
    )
    config=ServerConfig(num_rounds=3)
    return ServerAppComponents(
        strategy=strategy,
        config=config,
    )

In [ ]:
server = ServerApp(server_fn=server_fn)

In [ ]:

run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=3,
    backend_config=backend_setup,
)